In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/DSBDA/ProjectDSBDA/final_dataset.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-9959a6d83120>:9: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/DSBDA/ProjectDSBDA/final_dataset.csv')


In [3]:
# looking at hypotesis we drop all the columns that aren't needed
# column needed: index, country (we keep this as geographical feature), awards(but we keep only the number -> num_awards), price_range (more accurate than price_level), cuisines, features, veget/vegan/gluten, open_hours_per_week
df=df.drop(columns=['restaurant_name', 'region', 'province', 'city', 'latitude', 'longitude', 'popularity_detailed', 'popularity_generic', 'claimed', 'awards', 'top_tags', 'price_level', 'meals', 'special_diets', 'original_open_hours', 'open_days_per_week', 'working_shifts_per_week'])

# keep only avg_rating as column for rating (?)
df=df.drop(columns=['total_reviews_count', 'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere'])

print(df.columns.tolist())
print(len(df.columns.tolist()))

['index', 'country', 'price_range', 'cuisines', 'features', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'open_hours_per_week', 'avg_rating', 'population,N,19,11', 'num_awards']
12


In [4]:
# same reasoning of awards with features and cuisines
df['num_features'] = df['features'].apply(lambda x: len(x.split(',')) if isinstance(x, str) and x else 0)
df['num_cuisines'] = df['cuisines'].apply(lambda x: len(x.split(',')) if isinstance(x, str) and x else 0)
df=df.drop(columns=['features', 'cuisines'])

print(df.columns.tolist())
print(len(df.columns.tolist()))

['index', 'country', 'price_range', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'open_hours_per_week', 'avg_rating', 'population,N,19,11', 'num_awards', 'num_features', 'num_cuisines']
12


In [5]:
# transform price_range in the mean of the range
df['avg_price'] = df['price_range'].apply(lambda x:
    (int(x.split('-')[0].replace('€', '').replace('CHF', '').replace('\xa0', '').replace(',', '')) +
     int(x.split('-')[1].replace('€', '').replace('CHF', '').replace('\xa0', '').replace(',', ''))) / 2
    if isinstance(x, str) and '-' in x else np.nan)

df=df.drop(columns=['price_range'])

print(df.columns.tolist())
print(len(df.columns.tolist()))

['index', 'country', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'open_hours_per_week', 'avg_rating', 'population,N,19,11', 'num_awards', 'num_features', 'num_cuisines', 'avg_price']
12


In [6]:
# correlation matrix

df = pd.get_dummies(df, columns=['country'], drop_first=True)           #one-hot vector

# replace Y/N with True/False
df['vegetarian_friendly'] = df['vegetarian_friendly'].replace({'Y': True, 'N': False})
df['vegan_options'] = df['vegan_options'].replace({'Y': True, 'N': False})
df['gluten_free'] = df['gluten_free'].replace({'Y': True, 'N': False})

# plot matrix
correlation_matrix = df.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,index,vegetarian_friendly,vegan_options,gluten_free,open_hours_per_week,avg_rating,"population,N,19,11",num_awards,num_features,num_cuisines,avg_price,country_Belgium,country_Bulgaria,country_Croatia,country_Czech Republic,country_Denmark,country_England,country_Finland,country_France,country_Germany,country_Greece,country_Hungary,country_Ireland,country_Italy,country_Northern Ireland,country_Poland,country_Portugal,country_Romania,country_Scotland,country_Slovakia,country_Spain,country_Sweden,country_The Netherlands,country_Wales
index,1.000000,0.009383,0.027315,0.047813,0.102201,0.034821,-0.294460,0.026530,-0.007819,0.113689,-0.000690,0.228639,0.093356,0.143838,0.161728,0.134900,-0.042887,0.117402,-0.530348,-0.350565,0.218862,0.143199,0.097325,0.383534,-0.003839,0.219727,0.231337,0.159595,-0.007435,0.085073,-0.300287,0.162602,-0.194277,-0.005062
vegetarian_friendly,0.009383,1.000000,0.551527,0.479455,-0.072409,0.138253,-0.025142,0.457982,-0.121298,0.339292,-0.002073,-0.008798,-0.007728,0.018294,-0.035535,-0.025674,0.103652,-0.019749,-0.096822,0.042650,0.012574,-0.013086,0.023345,0.049156,0.010642,-0.064057,-0.031286,-0.026159,0.039160,-0.020854,-0.062058,-0.032750,0.036789,0.027473
vegan_options,0.027315,0.551527,1.000000,0.546782,-0.040107,0.143191,-0.006742,0.434749,-0.007464,0.247762,-0.001280,-0.027243,-0.001047,0.006344,-0.023431,-0.014397,0.129914,-0.009050,-0.078475,-0.037244,0.048517,-0.008499,0.030669,-0.006835,0.015584,-0.037081,-0.010573,-0.011443,0.048837,-0.013088,0.000154,-0.016494,-0.000827,0.028733
gluten_free,0.047813,0.479455,0.546782,1.000000,-0.069725,0.125266,-0.031813,0.554182,0.020687,0.267879,-0.000752,-0.031972,-0.009387,0.002584,-0.025623,-0.018164,0.132154,-0.010529,-0.105932,-0.074302,0.005635,-0.014027,0.035663,0.042793,0.018202,-0.049063,-0.006041,-0.022576,0.054542,-0.014309,0.018679,-0.019637,-0.000498,0.032550
open_hours_per_week,0.102201,-0.072409,-0.040107,-0.069725,1.000000,-0.166675,0.015832,-0.124431,0.050012,-0.030955,0.007971,-0.072151,0.037298,0.073974,0.036343,-0.007083,-0.000748,-0.000015,-0.121172,-0.009747,0.129145,0.024900,0.001156,-0.044662,-0.003234,0.088931,0.015203,0.066324,-0.003113,0.019002,0.051595,-0.015302,-0.013084,-0.012519
avg_rating,0.034821,0.138253,0.143191,0.125266,-0.166675,1.000000,-0.022313,0.195133,-0.060439,0.092158,-0.007226,-0.008497,0.012348,0.015177,0.001095,-0.011550,-0.018685,-0.009236,-0.006896,0.012953,0.063477,0.014248,0.012857,0.009284,-0.003684,0.019362,0.025096,0.006693,0.001991,0.006158,-0.047821,-0.030537,0.005950,0.003484
"population,N,19,11",-0.294460,-0.025142,-0.006742,-0.031813,0.015832,-0.022313,1.000000,-0.012522,-0.011063,-0.045298,-0.001491,-0.056797,0.068406,-0.017459,0.011131,0.002120,0.003093,-0.000941,0.470865,0.002309,0.006821,0.089905,0.028576,-0.234816,-0.013469,-0.067357,0.006866,-0.025604,-0.029090,0.018442,0.039561,0.014232,0.054408,0.012511
num_awards,0.026530,0.457982,0.434749,0.554182,-0.124431,0.195133,-0.012522,1.000000,0.086990,0.273936,-0.000957,-0.020308,-0.011470,0.013563,-0.021930,-0.013287,0.050459,-0.020413,0.018048,-0.085422,0.031054,-0.013701,0.020570,0.064373,0.005018,-0.040055,0.004951,-0.027328,0.036071,-0.016042,-0.027754,-0.035643,-0.017219,0.016566
num_features,-0.007819,-0.121298,-0.007464,0.020687,0.050012,-0.060439,-0.011063,0.086990,1.000000,0.080451,0.000088,0.003259,-0.006108,0.012681,-0.008585,-0.009528,0.016810,-0.013772,0.058691,-0.076958,0.002401,-0.012023,0.006121,0.011266,0.000713,-0.042890,0.024608,-0.004453,0.016500,-0.002382,0.019361,-0.030743,-0.027834,0.007900
num_cuisines,0.113689,0.339292,0.247762,0.267879,-0.030955,0.092158,-0.045298,0.273936,0.080451,1.000000,-0.002488,0.009366,0.007046,0.060672,-0.010019,-0.013886,-0.038037,-0.017228,-0.075454,-0.087588,0.015816,0.007733,0.002049,0.127822,-0.006370,-0.019653,0.033445,0.022134,0.010653,-0.014104,-0.020340,-0.035296,0.019610,-0.003426


In [7]:
def find_max_value(matrix):
    max_value = None
    max_idx = None
    n = matrix.shape[0]
    for i in range(n):
        for j in range(i + 1, n):
            if pd.notna(matrix.iloc[i, j]):
                if max_value is None or matrix.iloc[i, j] > max_value:
                    max_value = matrix.iloc[i, j]
                    max_idx = (matrix.index[i], matrix.columns[j])
    return max_value, max_idx

def find_min_value(matrix):
    min_value = None
    min_idx = None
    n = matrix.shape[0]
    for i in range(n):
        for j in range(i + 1, n):
            if pd.notna(matrix.iloc[i, j]):
                if min_value is None or matrix.iloc[i, j] < min_value:
                    min_value = matrix.iloc[i, j]
                    min_idx = (matrix.index[i], matrix.columns[j])
    return min_value, min_idx

max_value, max_idx = find_max_value(correlation_matrix)
print(f"Max: {max_value} at {max_idx}")

min_value, min_idx = find_min_value(correlation_matrix)
print(f"Min: {min_value} at {min_idx}")

Max: 0.5541822329538166 at ('gluten_free', 'num_awards')
Min: -0.5303480442368538 at ('index', 'country_France')
